In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.linalg as la
np.set_printoptions(precision=2, formatter={'float':lambda x: f"{x:5.02f}"})

In [34]:
import numpy as np
def createBox():
    pts = [
        [0,0,0],
        [0,0,1],
        [0,1,0],
        [1,0,0],
        [1,1,0],
        [0,1,1],
        [1,0,1],
        [1,1,1],
        ]
    return np.array(pts).astype("float32")

box = createBox()
box = box + [50,50,50]
box

array([[50.00, 50.00, 50.00],
       [50.00, 50.00, 51.00],
       [50.00, 51.00, 50.00],
       [51.00, 50.00, 50.00],
       [51.00, 51.00, 50.00],
       [50.00, 51.00, 51.00],
       [51.00, 50.00, 51.00],
       [51.00, 51.00, 51.00]])

In [43]:
# basic pinhole camera
K = [
    [100, 0, 50],
    [0, 100, 50],
    [0,0,1]
    ]
K = np.array(K).astype("float32")

E = [
    [1,0,0,1],
    [0,1,0,1],
    [0,0,1,1],
    [0,0,0,1]
]
E = np.array(E)

E = la.inv(E)[:3,:]

P_native = np.matmul(K,E)
P_native

array([[100.00,  0.00, 50.00, -150.00],
       [ 0.00, 100.00, 50.00, -150.00],
       [ 0.00,  0.00,  1.00, -1.00]])

In [44]:
# view box

box_hat = np.hstack((box,np.ones((box.shape[0],1))))

box_x_hat = np.matmul(P_native,box_hat.T).T
box_x = np.array([box_x_hat[i,:] / box_x_hat[i,-1] for i in range(box_x_hat.shape[0])])[:,:-1]
box_x

array([[150.00, 150.00],
       [148.00, 148.00],
       [150.00, 152.04],
       [152.04, 150.00],
       [152.04, 152.04],
       [148.00, 150.00],
       [150.00, 148.00],
       [150.00, 150.00]])

In [45]:
from src.pycv.perspective import calibrate_dlt, decompose_perspective_projection_matrix

P = calibrate_dlt(box_x, box)

K, R, T = decompose_perspective_projection_matrix(P)
print(f"{K}\n\n{R}\n\n{T}")

[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]

[[-1.00  0.00  0.00]
 [-0.00 -1.00 -0.00]
 [ 0.00 -0.00  1.00]]

[ 1.00  1.00  1.00]


In [39]:
box2 = createBox()
box2 = box2 + [5,5,5]
box2

array([[ 5.00,  5.00,  5.00],
       [ 5.00,  5.00,  6.00],
       [ 5.00,  6.00,  5.00],
       [ 6.00,  5.00,  5.00],
       [ 6.00,  6.00,  5.00],
       [ 5.00,  6.00,  6.00],
       [ 6.00,  5.00,  6.00],
       [ 6.00,  6.00,  6.00]])

In [30]:

P2 = calibrate_dlt(box_x, box)

K2, R2, T2 = decompose_perspective_projection_matrix(P)
print(f"{K}\n\n{R}\n\n{T}")

error: OpenCV(4.5.5) /io/opencv/modules/calib3d/src/calibration.cpp:3350: error: (-210:Unsupported format or combination of formats) objectPoints should contain vector of vectors of points of type Point3f in function 'collectCalibrationData'


In [47]:
from src.pycv.perspective import rot2eul, eul2rot

rot2eul(-1*np.eye(3))

array([-3.14,  0.00, -3.14])